In [16]:
import re
import json
import pandas as pd
# import datetime
import re
from collections import Counter
import numpy as np
import time
import math
import sys
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import editdistance
import copy
quiz = pd.read_csv(filepath_or_buffer= "C:\\Users\\zhazhang\\Desktop\\GitHub\\quizzResult.csv")
data1 = pd.read_csv(filepath_or_buffer="C:\\Users\\zhazhang\\Desktop\\GitHub\\event_semester1.csv")
exam = pd.read_csv(filepath_or_buffer="C:\\Users\\zhazhang\\Desktop\\GitHub\\exam_result.csv")
data1 = (data1[data1['action'].isin(['Viewed'])])
quiz.score = quiz.score.map(lambda x: x*10)
exam['score'] = exam.score.map(lambda x: (x/18*100))
quiz['action'] = "Submitted"
event = pd.concat([data1[['userId','objectId','action','Timestamp']],quiz[['userId','objectId','action','Timestamp']]])
exam['action']='FinalExam'
exam['Timestamp'] = 1542835584754#1542835584754 # this is the right timestamp of exam
exam['objectId'] = 11209 # I faked an
#event = pd.concat([event, exam[['userId','objectId','action','Timestamp']]])

"""
Here is the treat of event which add the correlated quiz and exam score 
"""
user_list = list(set(event['userId']))
next_score = []
quiz1 = list(set(quiz.objectId))[0]
quiz2 = list(set(quiz.objectId))[1]
exam_time = exam.iloc[0]['Timestamp']
user_quized = []
# print(len(user_list))
for i in user_list:
    if ((len(quiz[quiz.userId==i])) == 2) & (i in list(exam.userId)):
        #quiz=quiz[quiz.userId!=i]
        user_quized.append(i)
#print(len(user_list))
event = event[event.userId.isin(user_quized)]
#print(len(event))
event = event[event.objectId != 23133]
event=event.sort_values(by='Timestamp', ascending=True)
# resourceIndex = {8935: 0, 8936: 1, 316752: 2, 321242: 3, 321240: 4, 321237:5, 11209:6 }
#l = list(resourceIndex.keys())
#event1 = event[event["objectId"] == int(l[0])]
#or event["objectId"] == int(l[1]) or event["objectId"] == int(l[2]) or event["objectId"] == int(l[3]) or event["objectId"] == int(l[4]) or event["objectId"] == int(l[5]) or event["objectId"] == int(l[6])or event["objectId"] == int(l[7])or event["objectId"] == int(l[8]) or event["objectId"] == int(l[9])]

# event2 = event[event["objectId"] == int(l[1])]
# event3 = event[event["objectId"] == int(l[2])]
# event4 = event[event["objectId"] == int(l[3])]
# event5 = event[event["objectId"] == int(l[4])]
# event6 = event[event["objectId"] == int(l[5])]
# event7 = event[event["objectId"] == int(l[6])]

# event8 = event[event["objectId"] == int(l[7])]
# event9 = event[event["objectId"] == int(l[8])]
# event10 = event[event["objectId"] == int(l[9])]
#event = pd.concat([event1,event2,event3,event4,event5,event6,event7])#,event8,event9,event10
s_good = [] #top 30%
s_normal = []# middle 40%
s_bad = []# next 30%

exam=exam.sort_values(by='score', ascending=False)
for i in range(len(exam)):
    if i < 30:
        s_good.append(exam.iloc[i]["userId"])

    if i>=30 and i<60:
        s_normal.append(exam.iloc[i]["userId"])

    if i>=70 and i<100:
        s_bad.append(exam.iloc[i]["userId"])
print("student level dividing done")

def get_key(dic, value):
    """
    know the value, get keys
    """
    a = [k for k, v in dic.items() if v == value]
    return a[0]

event = event[event.objectId != 23133]
event_c = event[event.Timestamp < 1542835584755]# course
event_r = event[event.Timestamp >=1542835584755]# review
l_good = []
l_bad = []
l_normal = []
for user in s_good:
    l_path = list(event_r[event_r.userId==user].objectId)
    l_good.append(l_path)
for user in s_normal:
    l_path = list(event_r[event_r.userId==user].objectId)
    l_normal.append(l_path)
for user in s_bad:
    l_path = list(event_r[event_r.userId==user].objectId)
    l_bad.append(l_path)
l_total = l_good+l_bad+l_normal

student level dividing done


In [37]:
s_good_novide = [] #top 30%
s_normal_novide = []# middle 40%
s_bad_novide = []# next 30%

exam=exam.sort_values(by='score', ascending=False)
for i in range(len(exam)):
    if i < 30 :
        try:
            if list(event_r[event_r.userId==exam.iloc[i]["userId"]].objectId)!=[]:
                s_good_novide.append(exam.iloc[i]["userId"])
        except IndexError:
            continue
    if i>=30 and i<60:
        try:
            if list(event_r[event_r.userId==exam.iloc[i]["userId"]].objectId)!=[]:
                s_normal_novide.append(exam.iloc[i]["userId"])
        except IndexError:
            continue
    if i>=70 and i<100:
        try:
            if list(event_r[event_r.userId==exam.iloc[i]["userId"]].objectId)!=[]:
                s_bad_novide.append(exam.iloc[i]["userId"])
        except IndexError:
            continue
print("student level dividing done")
l_good_novide = []
l_bad_novide = []
l_normal_novide = []
for user in s_good:
    l_path = list(event_r[event_r.userId==user].objectId)
    if l_path !=[]:
        l_good_novide.append(l_path)
for user in s_normal:
    l_path = list(event_r[event_r.userId==user].objectId)
    if l_path !=[]:
        l_normal_novide.append(l_path)
for user in s_bad:
    l_path = list(event_r[event_r.userId==user].objectId)
    if l_path !=[]:
        l_bad_novide.append(l_path)

student level dividing done


In [38]:
len(s_normal_novide)

13

In [111]:
from prefixspan import PrefixSpan
#resourcelist = list(set(event.objectId))
resourcelist = [8995, 8996, 325071, 318151, 328647, 316747, 316748, 316749, 316750, 316751, 316752, 325068, 325070, 325069, 325072, 321237, 321238, 321239, 321240, 321241, 321242, 330708, 330711, 319715, 8935, 8936, 32245, 322555]
print("real LP",(list(event_r[event_r.userId==s_good_novide[0]].objectId)))
def spm(student = s_good_novide[0]):
    event=event_r
    l_target = list(event_r[event_r.userId==student].objectId)
    path_length = len(l_target)
    event = event[event.userId !=student]
    userlist = list(set(event.userId))
    trainingset = []
    for user in userlist:
        l_path = list(event[event.userId==user].objectId)
        trainingset.append(l_path)
    print("student is: ",student)
    #trainingset = trainingset+l_good[:11]
    
    # Here we put the influence or this stdent's learning log bigger. x10
    if list(event_r[event_r.userId==student].objectId) in trainingset:
        trainingset.remove(list(event_r[event_r.userId==student].objectId))
    for i in range(10):
        trainingset.append(l_target)
    ps = PrefixSpan(trainingset)
    
    pattern = ps.topk(1000, filter=lambda patt, matches: len(patt)>2)# pattern lenth should bigger than 1
    pattern_time = {} #Here stores all pattern with appear times
    
    for i,element in enumerate(pattern):
        l_s = []# store pattern in this element
        s = ""
        for i in range(len(element[1])):
            if i==0:
                s=str(element[1][i])
            else:
                l_s.append(s+","+str(element[1][i]))
                s = str(element[1][i])
        for j in l_s:
            if j in pattern_time.keys():
                pattern_time[j]+=element[0]
            else:
                pattern_time[j]=element[0]
           
    # ordered pattern in list            
    pattern_time = sorted(pattern_time.items(),key = lambda pattern_time:pattern_time[1],reverse=True)
    
    # delete repeat part
    #print(len(pattern_time))
    
    """
    Here is deduplication.
    we can't delete the item of list in for cycle. It will have 'index out of range problem'. 
    So we store the repeat index and delete after
    """ 
    delete_indice = [] 
    for k1 in range(len(pattern_time)):
        starter = pattern_time[k1][0].split(",")[0]
        ender = pattern_time[k1][0].split(",")[1]
        if starter == ender:
            delete_indice.append(k1)
        if pattern_time[k1]==pattern_time[-1]:
            break
            
        for k2 in range(k1+1,len(pattern_time)):
            #print(pattern_time[k2])
            temps_start = pattern_time[k2][0].split(",")[0]
            temps_end = pattern_time[k2][0].split(",")[1]
            if starter == temps_start:
                delete_indice.append(pattern_time[k2])
            if ender == pattern_time[k2][0].split(",")[1]:
                delete_indice.append(pattern_time[k2])    
                
#     for  i in set(delete_indice):
#         if i in pattern_time:
#             pattern_time.remove(i)
#     print("pattern with frequence:",pattern_time)
        
    """
    Here we organise the path from pattern list.
    We should firstly find the head then finish the path.
    """
    element = []
       
    pattern_result = [x[0] for x in pattern_time] # delete pattern show times, keep pattern
    print("unique pattern:",pattern_result)  
    store = []
    for i in range(len(pattern_result)):
        for j in range(len(pattern_result)):
            if i==j:
                continue
            if pattern_result[i].split(",")[0] in pattern_result[j]:
                store.append(pattern_result[i])
#     path =list(set(pattern_result).difference(set(store)))[0] #这里的是删除了不同
#     print("begin_node of path:",path)
    #print(event[event.userId==student]['objectId'])
    
    path = str(l_target[0])
    compt=0
    c_b = 0
    l_change = 2
    repeat_times = 0
    used_pattern = {}
    while compt < path_length-1: # First node has two element, so we should add path_length-2
        c_b+=1
        for i in pattern_result:
#             if i.split(",")[0]==i.split(",")[-1]:# delete repeat in pattern，如 [1,1]
#                 continue
            if i in used_pattern.keys():
                if used_pattern[i]>1:
                    continue
                
            if ',' in path:
                if i.split(",")[0]==path.split(",")[-1]:
                    path+=","+i.split(",")[-1]
                    compt+=1
                    if i in used_pattern.keys():
                        used_pattern[i]+=1
                    else:
                        used_pattern[i]=1
            else:
                if i.split(",")[0]==path:
                    path+=","+i.split(",")[-1]
                    compt+=1
                    used_pattern[i]=1
        if l_change==len(path):
            c_b+=1
        else:
            l_change=len(path)
        if c_b>100:
            path+=","+i
            print("i:",i)
            compt+=2
            if i in used_pattern.keys():
                used_pattern[i]+=1
            else:
                used_pattern[i]=1
    print("recommandation:",path)
    return path 

path = spm()

real LP [327938, 333191, 333191, 328647, 327959]
student is:  wolter6u
unique pattern: ['321240,321241', '321241,325071', '327940,330708', '325071,325072', '325071,327940', '316751,321240', '325072,327940', '330708,333190', '321240,325071', '316750,321240', '321240,321240', '316750,316751', '327940,327941', '330708,330708', '321241,325072', '327941,330708', '325071,330708', '327940,327940', '321241,327940', '316751,321241', '321240,327940', '327940,333190', '316750,321241', '325072,330708', '325071,325071', '325072,327941', '321240,325072', '321241,330708', '321240,330708', '316750,325071', '316751,325071', '325071,333190', '327941,333190', '325072,325072', '325071,327941', '316750,316750', '316750,327940', '316751,327940', '316750,325072', '321240,333190', '321241,327941', '325072,333190', '321241,333190', '333191,333191', '321241,321240', '327938,333191', '333191,328647', '316751,330708', '316751,316751', '316751,325072', '328647,327959', '321241,321241', '325071,321240', '327940,328

664

In [112]:
result_SPM_noraml = []
for i,s in enumerate(s_normal_novide):
    result_SPM_noraml.append(spm(s))
    
result_SPM_bad = []
for i,s in enumerate(s_bad_novide):
    result_SPM_bad.append(spm(s))
    
result_SPM_good = []
for i,s in enumerate(s_good_novide):
    result_SPM_good.append(spm(s))

student is:  vignot1u
unique pattern: ['321240,321241', '321241,325071', '325072,327940', '325071,325072', '327940,327941', '327940,330708', '327941,330708', '325071,327940', '316751,321240', '316750,316751', '330708,333190', '321240,325071', '321241,325072', '327940,327940', '316750,321240', '325072,327941', '316751,321241', '321241,327940', '327940,333190', '321240,325072', '325072,325072', '325072,330708', '316750,321241', '325071,330708', '316751,325071', '321240,327940', '325071,327941', '327941,333190', '316750,325071', '330708,330711', '330711,333190', '327941,327940', '321241,330708', '321241,327941', '316750,325072', '327941,333191', '325071,333190', '316751,327940', '325072,333190', '316751,325072', '327940,333191', '316750,327940', '321240,330708', '321240,327941', '321241,333190', '327959,327960', '325072,333191', '325071,330711', '321240,333190', '327938,327959', '327940,330711', '321240,321240', '325071,333191', '316751,330708', '327938,327960', '333190,321240', '316750,3

unique pattern: ['321240,321241', '321241,325071', '327940,330708', '330708,333190', '321240,321240', '325071,325072', '325071,327940', '325072,327940', '316751,321240', '316750,321240', '321240,325071', '316750,316751', '327940,327941', '330708,330708', '325071,330708', '327941,330708', '321241,325072', '327940,327940', '321241,327940', '316751,321241', '321240,327940', '327940,333190', '316750,321241', '321240,330708', '325072,330708', '325071,325071', '325072,327941', '321240,325072', '321241,330708', '316750,325071', '316751,325071', '325071,333190', '327941,333190', '333190,333190', '321240,333190', '325072,325072', '325071,327941', '316750,316750', '316750,327940', '316751,327940', '316750,325072', '325072,333190', '321241,333190', '321241,327941', '321241,321240', '316751,330708', '316750,330708', '316751,316751', '316751,325072', '321241,321241', '325071,321240', '327940,328647', '321240,316751', '321240,327941', '316751,316750', '316750,333190', '325071,328647', '325071,333191

unique pattern: ['321240,321240', '321240,327940', '321240,316750', '327940,321240', '327959,327960', '325068,327959', '316750,333191', '333191,327938', '327938,325068', '321240,321241', '321241,325071', '321240,333191', '327940,316750', '327938,327959', '316750,327938', '327940,330708', '325068,327960', '333191,325068', '325071,327940', '316751,321240', '321240,325071', '321240,327938', '316750,321240', '327940,333191', '325071,325072', '330708,333190', '316750,316751', '327938,327960', '325071,330708', '316750,325068', '325072,327940', '333191,327959', '321241,327940', '321241,325072', '327940,333190', '330708,330708', '327940,327941', '321240,325068', '327940,327938', '316751,321241', '327941,330708', '316750,327959', '316750,321241', '333191,327960', '325071,333190', '321240,330708', '321241,330708', '327940,327940', '316750,325071', '325072,330708', '321240,325072', '321240,327959', '327940,325068', '325072,327941', '325071,325071', '316751,325071', '316750,327960', '321241,333190

unique pattern: ['321240,321241', '321241,325071', '327940,330708', '325071,325072', '325071,327940', '316751,321240', '325072,327940', '330708,333190', '321240,325071', '316750,321240', '321240,321240', '316750,316751', '327940,327941', '327941,330708', '330708,330708', '321241,325072', '325071,330708', '327940,327940', '321241,327940', '316751,321241', '321240,327940', '327940,333190', '316750,321241', '325072,330708', '325071,325071', '325072,327941', '321240,325072', '321241,330708', '321240,330708', '316750,325071', '316751,325071', '325071,333190', '327941,333190', '325072,325072', '325071,327941', '316750,316750', '316750,327940', '316751,327940', '325072,333190', '316750,325072', '321240,333190', '321241,327941', '321241,333190', '321241,321240', '316751,330708', '316751,316751', '316751,325072', '321241,321241', '325071,321240', '327940,328647', '321240,316751', '316750,330708', '321240,327941', '333190,333190', '316751,316750', '325071,328647', '325071,333191', '325072,333191

student is:  lods3u
unique pattern: ['321240,321241', '321241,325071', '327940,330708', '325071,325072', '325071,327940', '316751,321240', '325072,327940', '330708,333190', '321240,325071', '316750,321240', '321240,321240', '316750,316751', '327940,327941', '327941,330708', '330708,330708', '321241,325072', '325071,330708', '327940,327940', '321241,327940', '316751,321241', '321240,327940', '327940,333190', '316750,321241', '325072,330708', '325071,325071', '325072,327941', '321240,325072', '321241,330708', '321240,330708', '316750,325071', '316751,325071', '325071,333190', '327941,333190', '325072,325072', '325071,327941', '316750,316750', '316750,327940', '316751,327940', '325072,333190', '316750,325072', '321240,333190', '321241,327941', '321241,333190', '321241,321240', '316751,330708', '316751,316751', '316751,325072', '321241,321241', '325071,321240', '327940,328647', '321240,316751', '316750,330708', '321240,327941', '333190,333190', '316751,316750', '325071,328647', '325071,333

unique pattern: ['321240,321241', '321241,325071', '316751,321240', '327940,330708', '316750,321240', '321240,321240', '325071,327940', '321240,325071', '325071,325072', '325072,327940', '330708,333190', '316750,316751', '327940,327941', '321240,325072', '325071,330708', '330708,330708', '316751,321241', '321241,325072', '327941,330708', '321241,327940', '327940,327940', '321240,327940', '327940,333190', '325072,330708', '321240,316750', '316750,321241', '325072,327941', '325071,325071', '325072,327959', '327959,327938', '321240,330708', '321241,330708', '316751,325071', '316751,316750', '316750,325072', '325071,333190', '316750,325071', '327941,333190', '321240,327959', '325071,327941', '325072,327938', '325072,325072', '316750,316750', '316751,325072', '316750,327940', '316751,327940', '321240,327938', '321240,333190', '316750,327959', '321241,333190', '321241,327941', '325072,333190', '316751,330708', '321241,321240', '316751,316751', '321240,316751', '316750,330708', '321240,327941

unique pattern: ['321241,325071', '321240,321241', '325071,327940', '316751,321240', '316750,321240', '327940,330708', '330708,333190', '316750,316751', '325071,330708', '321240,325071', '325071,325071', '327940,333190', '316751,321241', '321241,327940', '325071,333190', '316751,316750', '333190,328647', '316751,316751', '316750,321241', '330708,328647', '321240,327940', '327940,325071', '316750,316750', '316751,325071', '333190,330708', '316750,325071', '333190,333190', '321241,330708', '325071,328647', '327940,328647', '321241,333190', '321240,330708', '316750,327940', '316751,327940', '321240,333190', '321240,321240', '316751,330708', '316750,330708', '321241,328647', '316751,333190', '316750,333190', '325071,325072', '321240,328647', '321241,325072', '330708,330708', '316751,328647', '321240,325072', '325072,327940', '325072,330708']
i: 325072,330708
i: 325072,330708
i: 325072,330708
recommandation: 316751,321240,325071,325071,333190,328647,325072,330708,333190,328647,325072,330708

In [61]:
def transition_diff(resourceIndex, df_total):
    print("calculating the transition matrix from all the records")
     
    n = len(resourceIndex)  # number of states
    M = np.zeros((n, n))  # Matrix for all the
    UserID = []  # 存储处理的用户的顺序。
    actionspace = []
    for i in set(df_total['userId']):
#         print("treating user :" + i)
        i_user = df_total[df_total['userId'].isin([i])]
        i_user = i_user.sort_values(by='Timestamp')
        UserID.append(i_user)
        listResource = []
        for j in i_user.objectId:
            listResource.append(resourceIndex[j])
        actionspace.append(listResource)  # 将进行过的每个用户的学习过程全部记录下来
        for (i, j) in zip(listResource, listResource[1:]):
            M[i][j] += 1

    # print(sum(M))
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f / s for f in row]
    return M

def get_key(dic, value):
    """
    know the value, get keys
    """
    a = [k for k, v in dic.items() if v == value]
    return a[0]

# 预测状态的马尔可夫模型的函数
state = [i for i in range(len(set(event["objectId"])))]
resourceIndex = {}
for i,e in  enumerate(set(event["objectId"])):
    resourceIndex[e]=i
print(resourceIndex)
def MC(student=s_good[0],trainingset=l_good):
    # define
#     print(event1[event1.userId==student])
    #M_s = transition_diff(resource, event2[event2.userId==student])
    resource=resourceIndex
    states=state
    event=event_r
    l_target = list(event[event.userId==student].objectId)
    resourceNow = resourceIndex[l_target[0]]
    path_length = len(l_target)
    event = event[event.userId !=student]
    userlist = list(set(event.userId))
    trainingset = []
    path_length=len(l_target)
    event_c
    event2=event_r
    #M_s = transition_diff(resource, event_c[event_c.userId==student])
    for i in range(10):
        event2 =  pd.concat([event2,event_c[event_c.userId==student]])
    T = transition_diff(resource, event2)

    activityList = [resourceNow]
    print("Start state: " + str(resourceNow))
    # 应该记录选择的状态序列。这里现在只有初始状态。
    
    i = 0
    # 计算 activityList 的概率
    prob = 1
    while i != path_length:
        try:
            change = np.random.choice(list(states), replace=True, p=T[resourceNow])
        except ValueError:
            return []
        activityList.append(change)
        prob = prob*T[resourceNow][change]
        resourceNow = change
        i += 1
    
    #print("End state after "+ str(resource) + " resource: " + str(resourceNow))
    print("Probability of the possible sequence of states: " + str(prob))
    activityList = [get_key(resourceIndex,x) for x in activityList]
    print("Possible states: ",(activityList))
    return (activityList)
a = MC(s_good[1])

{327938: 0, 327940: 1, 327941: 2, 333190: 3, 333191: 4, 327959: 5, 327960: 6, 327966: 7, 327967: 8, 8995: 9, 8996: 10, 325071: 11, 318151: 12, 328647: 13, 316747: 14, 316748: 15, 316749: 16, 316750: 17, 316751: 18, 316752: 19, 325068: 20, 325070: 21, 325069: 22, 325072: 23, 321237: 24, 321238: 25, 321239: 26, 321240: 27, 321241: 28, 321242: 29, 330708: 30, 330711: 31, 319715: 32, 8935: 33, 8936: 34, 32245: 35, 322555: 36}
calculating the transition matrix from all the records
Start state: 17
Probability of the possible sequence of states: 3.7144109121954916e-23
Possible states:  [316750, 321240, 321240, 321240, 325071, 325071, 325072, 327959, 321239, 321237, 321238, 321240, 316750, 316752, 316752, 316752, 318151, 322555, 321237, 321237, 325070, 325069, 325071, 325071, 327966, 327938, 316747, 316747, 316748, 319715, 8935, 8935, 8935]


In [174]:
result_MC_noraml = []
for i,s in enumerate(s_normal_novide):
    result_MC_noraml.append(MC(student = s))
    
result_MC_bad = []
for i,s in enumerate(s_bad_novide):
    result_MC_bad.append(MC(student = s))
result_MC_good = []
for i,s in enumerate(s_good_novide):
    result_MC_good.append(MC(student = s))    

calculating the transition matrix from all the records
Start state: 4
Probability of the possible sequence of states: 1.047522934465913e-21
Possible states:  [333191, 333190, 333190, 330711, 330711, 333190, 333191, 333191, 327967, 333190, 333191, 333191, 333191, 333191, 333191, 328647, 322555, 322555, 321237, 321237, 321237, 321237, 321237, 321238, 321238, 321238, 321238, 325070, 325070, 327960, 327960, 327960, 321240, 321240, 321241, 321241, 325071, 327940]
calculating the transition matrix from all the records
Start state: 17
Probability of the possible sequence of states: 7.4109901679788705e-06
Possible states:  [316750, 316750, 316751, 321240, 321241, 325071, 325072, 327966, 327940, 327940]
calculating the transition matrix from all the records
Start state: 17
Probability of the possible sequence of states: 8.013590576198958e-11
Possible states:  [316750, 321240, 321240, 321240, 327941, 327940, 327941, 327940, 327941, 330708, 333190, 333190, 330711, 330711, 330711, 330711]
calculat

calculating the transition matrix from all the records
Start state: 17
Probability of the possible sequence of states: 1.81125627810389e-12
Possible states:  [316750, 316750, 321240, 321240, 321241, 321242, 325071, 325072, 316750, 316751, 333190, 333191, 330708, 333190, 319715, 316747, 316747, 316747]
calculating the transition matrix from all the records
Start state: 17
Probability of the possible sequence of states: 0.08853769992383853
Possible states:  [316750, 321240, 321241]
calculating the transition matrix from all the records
Start state: 0
Probability of the possible sequence of states: 3.823064205900421e-13
Possible states:  [327938, 327938, 327938, 328647, 328647, 328647, 328647, 328647, 327938, 327959, 327960, 327941, 330708, 330708, 327940, 330708, 327941, 330708, 330708, 330708, 333190]
calculating the transition matrix from all the records
Start state: 18
Probability of the possible sequence of states: 4.199127831911231e-06
Possible states:  [316751, 316752, 318151, 3225

## First measure GSLP

In [70]:

# SSLP calcule bad in SPM
distance = {}
len_path = {}
truth = l_good_novide+l_normal_novide
for i_c,e_c in enumerate(result_MC_bad):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    for j in truth: # truth grand
        d = editdistance.eval(str(e_c), str(j))
        if d<=min_d:
            min_d = d
            distance[i_c]=[len(e_c),len(j),d]
            len_path[i_c] = [e_c,j]
#good students

In [73]:
distance

{0: [11, 12, 46],
 1: [40, 32, 162],
 2: [54, 37, 209],
 3: [7, 5, 21],
 4: [12, 12, 33],
 5: [0, 1, 6],
 6: [40, 32, 149],
 7: [12, 12, 38],
 8: [2, 1, 8],
 9: [11, 11, 36],
 10: [2, 1, 8],
 11: [0, 1, 6],
 12: [16, 15, 53],
 13: [26, 19, 89],
 14: [6, 5, 15],
 15: [33, 32, 126],
 16: [8, 7, 25],
 17: [95, 94, 316],
 18: [10, 9, 39],
 19: [18, 17, 51],
 20: [3, 2, 11],
 21: [21, 17, 77],
 22: [14, 13, 32],
 23: [3, 2, 10],
 24: [14, 13, 39],
 25: [6, 5, 12],
 26: [17, 17, 54],
 27: [12, 13, 30],
 28: [18, 16, 61]}

In [72]:
len_path[]

{0: [[321240,
   321241,
   327940,
   327941,
   319715,
   316747,
   316747,
   316747,
   316747,
   316747,
   316747],
  [316750,
   321240,
   316751,
   321240,
   321241,
   325071,
   325071,
   330708,
   330711,
   330708,
   330708,
   333190]],
 1: [[321240,
   321241,
   321241,
   325071,
   325071,
   321241,
   321241,
   325071,
   325071,
   325072,
   325072,
   327940,
   327940,
   327940,
   330708,
   327940,
   327941,
   330708,
   330708,
   330711,
   327967,
   333190,
   333190,
   316749,
   316752,
   316752,
   322555,
   322555,
   322555,
   322555,
   321237,
   321237,
   321237,
   321238,
   321238,
   321239,
   321239,
   321239,
   321239,
   333190],
  [316750,
   319715,
   316747,
   322555,
   321237,
   328647,
   325068,
   327938,
   316749,
   316748,
   321238,
   321239,
   325068,
   325070,
   325069,
   327959,
   327960,
   333191,
   327938,
   316747,
   325068,
   327938,
   333191,
   321237,
   316747,
   327959,
   325070,


In [ ]:
print("results:",(30/6)+ 30/1.2/6       )

In [74]:
#MC
# SSLP calcule bad in MC
distance = {}
len_path = {}
truth = l_normal_novide
for i_c,e_c in enumerate(result_MC_bad):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    for j in truth: # truth grand
        d = editdistance.eval(str(e_c), str(j))
        if d<=min_d:
            min_d = d
            distance[i_c]=[len(e_c),len(j),d]
            len_path[i_c] = [e_c,j]

In [75]:
distance

{0: [11, 12, 46],
 1: [40, 37, 169],
 2: [54, 37, 209],
 3: [7, 7, 22],
 4: [12, 12, 33],
 5: [0, 1, 6],
 6: [40, 37, 161],
 7: [12, 12, 38],
 8: [2, 1, 8],
 9: [11, 9, 46],
 10: [2, 1, 8],
 11: [0, 1, 6],
 12: [16, 15, 53],
 13: [26, 19, 89],
 14: [6, 7, 27],
 15: [33, 37, 129],
 16: [8, 7, 29],
 17: [95, 37, 493],
 18: [10, 9, 46],
 19: [18, 17, 51],
 20: [3, 4, 15],
 21: [21, 17, 77],
 22: [14, 15, 36],
 23: [3, 4, 15],
 24: [14, 15, 43],
 25: [6, 7, 20],
 26: [17, 17, 54],
 27: [12, 12, 32],
 28: [18, 17, 62]}

In [76]:
# (15-5)*2/6 is the distance for BS AS, final result should be  (15)/2 = 3.7 
[6, 5, 15]
[3, 4, 15]

[3, 4, 15]

In [95]:
#SPM
# SSLP calcule bad in SPM
distance = {}
len_path = {}
truth = l_normal_novide
for i_c,e_c in enumerate(result_SPM_bad):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    for j in truth: # truth grand
        d = editdistance.eval(str(e_c), str(j))
        if d<=min_d:
            min_d = d
            distance[i_c]=[len(e_c),len(j),d]
            len_path[i_c] = [e_c,j]

In [80]:
len(result_SPM_bad)

14

In [96]:
distance

{0: [69, 9, 36],
 1: [272, 19, 174],
 2: [370, 37, 242],
 3: [69, 7, 39],
 4: [97, 12, 59],
 5: [5, 1, 7],
 6: [272, 19, 174],
 7: [97, 12, 59],
 8: [6, 1, 2],
 9: [69, 9, 36],
 10: [6, 1, 2],
 11: [5, 1, 7],
 12: [111, 12, 59],
 13: [195, 19, 117]}

In [125]:
#SPM
# SSLP calcule bad in SPM
distance = {}
len_path = {}
truth = l_good_novide
for i_c,e_c in enumerate(result_SPM_good):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    for j in truth: # truth grand
        d = editdistance.eval(str(e_c), str(j))
        if d<=min_d:
            min_d = d
            distance[i_c]=[int(len(e_c)/7),len(j),d-len(j)]
            len_path[i_c] = [e_c,j]

In [126]:
distance

{0: [4, 5, 12],
 1: [32, 17, 129],
 2: [7, 7, 18],
 3: [94, 94, 338],
 4: [19, 16, 57],
 5: [19, 17, 60],
 6: [6, 5, 22],
 7: [21, 17, 77],
 8: [16, 11, 56],
 9: [2, 2, 8],
 10: [13, 13, 37],
 11: [10, 11, 29],
 12: [15, 13, 47],
 13: [10, 11, 29],
 14: [17, 16, 52]}

In [130]:
#MC
# SSLP calcule good in MC
distance = {}
len_path = {}
truth = l_good_novide
for i_c,e_c in enumerate(result_MC_good):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    for j in truth: # truth grand
        d = editdistance.eval(str(e_c), str(j))
        if d<=min_d:
            min_d = d
            distance[i_c]=[int(len(e_c)),len(j),d]
            len_path[i_c] = [e_c,j]

In [131]:
distance

{0: [6, 5, 21],
 1: [33, 32, 130],
 2: [8, 7, 28],
 3: [95, 94, 330],
 4: [10, 9, 30],
 5: [18, 16, 52],
 6: [3, 2, 8],
 7: [21, 20, 71],
 8: [14, 13, 37],
 9: [3, 2, 10],
 10: [14, 13, 54],
 11: [6, 5, 15],
 12: [17, 17, 44],
 13: [12, 11, 32],
 14: [18, 17, 64]}

## Second measure SSLP

In [142]:
#MC
# SSLP calcule bad in MC
distance = {}
len_path = {}
truth = l_good_novide+l_normal_novide
for i_c,e_c in enumerate(l_bad_novide):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    for s_id,j in enumerate(truth): # truth grand
        d = editdistance.eval(str(e_c), str(j))
        if d<=min_d:
            min_d = d
            distance[i_c]=[int(len(e_c)),len(j),int(d/(len(e_c)/10)), {"s_id":s_id}]
            len_path[i_c] = [e_c,j]

### 从坏学生开始SSLP

In [145]:
print("profile select for BAD")
distance

profile select for BAD


{0: [10, 7, 34, {'s_id': 2}],
 1: [39, 37, 40, {'s_id': 15}],
 2: [53, 37, 31, {'s_id': 15}],
 3: [6, 5, 33, {'s_id': 11}],
 4: [11, 12, 18, {'s_id': 25}],
 5: [1, 1, 50, {'s_id': 20}],
 6: [39, 37, 40, {'s_id': 15}],
 7: [11, 12, 18, {'s_id': 25}],
 8: [1, 1, 0, {'s_id': 20}],
 9: [10, 7, 34, {'s_id': 2}],
 10: [1, 1, 0, {'s_id': 20}],
 11: [1, 1, 50, {'s_id': 20}],
 12: [15, 17, 19, {'s_id': 14}],
 13: [25, 20, 32, {'s_id': 7}]}

In [ ]:
#序号，长度
4:25 [11, 12]
7:25 [11, 12]
12:14 [15, 17] 19+18+18   

In [335]:
#MC
# SSLP calcule bad in MC
d_bad_sspl_MC = []
len_path = {}
duibi = [result_MC_bad[4],result_MC_bad[7],result_MC_bad[12]]
truth = l_good_novide+l_normal_novide
t = [truth[25],truth[25],truth[14]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_bad_sspl_MC.append(editdistance.eval(str(e_c), str(t[i_c])))
print(d_bad_sspl_MC)

[31, 44, 80]


In [336]:
print("d_bad_sspl_MC:",sum([31/12/40*10, 36/12/40*10, 69/17/40*10])/3)

d_bad_sspl_MC: 0.8035130718954249


In [338]:
print("distance SPM",sum([38/12/40*10, 69/17/40*10])/2)

distance SPM 0.9031862745098038


In [180]:
#SPM
# SSLP calcule bad in SPM
d_bad_sspl_SPM = []
len_path = {}
duibi = [result_SPM_bad[4],result_SPM_bad[7],result_SPM_bad[12]]
truth = l_good_novide+l_normal_novide
t = [truth[25],truth[25],truth[14]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_bad_sspl_SPM.append(editdistance.eval(str(e_c), str(t[i_c]))-len(t[i_c]))
print(d_bad_sspl_SPM)

[47, 47, 52]


In [166]:
print("d_bad_sspl_SPM:",sum([47/12/40*10, 47/12/40*10, 52/17/40*10])/3)

d_bad_sspl_SPM: 0.9076797385620914


In [332]:
sum([47/12/40*10, 47/12/40*10])/2

0.9791666666666666

#### 这里是不计算 GS AS的，单独对比GS 的 SSLP

In [168]:
#MC
# SSLP calcule bad in MC
distance = {}
len_path = {}
truth = l_good_novide
for i_c,e_c in enumerate(l_bad_novide):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    for s_id,j in enumerate(truth): # truth grand
        d = editdistance.eval(str(e_c), str(j))
        if d<=min_d:
            min_d = d
            distance[i_c]=[int(len(e_c)),len(j),int(d/(len(e_c)/10)), {"s_id":s_id}]
            len_path[i_c] = [e_c,j]
print("profile select for BAD")
distance

profile select for BAD


{0: [10, 7, 34, {'s_id': 2}],
 1: [39, 32, 41, {'s_id': 1}],
 2: [53, 32, 41, {'s_id': 1}],
 3: [6, 5, 33, {'s_id': 11}],
 4: [11, 13, 29, {'s_id': 8}],
 5: [1, 2, 100, {'s_id': 6}],
 6: [39, 32, 41, {'s_id': 1}],
 7: [11, 13, 29, {'s_id': 8}],
 8: [1, 2, 90, {'s_id': 10}],
 9: [10, 7, 34, {'s_id': 2}],
 10: [1, 2, 90, {'s_id': 10}],
 11: [1, 2, 100, {'s_id': 6}],
 12: [15, 17, 19, {'s_id': 14}],
 13: [25, 20, 32, {'s_id': 7}]}

In [ ]:
12: [15, 17, 19, {'s_id': 14}],
7: [11, 13, 29, {'s_id': 8}],
4: [11, 13, 29, {'s_id': 8}],

In [176]:
#MC
# SSLP calcule bad in MC
d_bad_sspl_SPM = []
len_path = {}
duibi = [result_SPM_bad[12],result_SPM_bad[7],result_SPM_bad[4]]
t = [truth[14],truth[8],truth[8]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_bad_sspl_SPM.append(editdistance.eval(str(e_c), str(t[i_c]))-len(t[i_c]))
print(d_bad_sspl_SPM)

[52, 56, 56]


In [177]:
print("d_bad_sspl_SPM without mentor in AS:",sum([52/16/40*10, 56/12/40*10, 56/12/40*10])/3)

d_bad_sspl_SPM without mentor in AS: 1.0486111111111112


In [179]:
#MC
# SSLP calcule bad in MC
d_bad_sspl_SPM = []
len_path = {}
duibi = [result_MC_bad[12],result_MC_bad[7],result_MC_bad[4]]
t = [truth[14],truth[8],truth[8]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_bad_sspl_SPM.append(editdistance.eval(str(e_c), str(t[i_c]))-len(t[i_c]))
print(d_bad_sspl_SPM)
print("d_bad_sspl_SPM without mentor in AS:",sum([63/16/40*10, 26/12/40*10, 23/12/40*10])/3)

[63, 26, 23]
d_bad_sspl_SPM without mentor in AS: 0.6684027777777777


### 这里是一般学生的 SSLP

In [182]:
#MC
# SSLP calcule bad in MC
distance = {}
len_path = {}
truth = l_good_novide
for i_c,e_c in enumerate(l_normal_novide):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    for s_id,j in enumerate(truth): # truth grand
        d = editdistance.eval(str(e_c), str(j))
        if d<=min_d:
            min_d = d
            distance[i_c]=[int(len(e_c)),len(j),int(d/(len(e_c)/10)), {"s_id":s_id}]
            len_path[i_c] = [e_c,j]
print("profile select for BAD")
distance

profile select for BAD


{0: [37, 32, 37, {'s_id': 1}],
 1: [9, 9, 33, {'s_id': 4}],
 2: [15, 16, 22, {'s_id': 12}],
 3: [4, 2, 40, {'s_id': 6}],
 4: [4, 2, 40, {'s_id': 6}],
 5: [1, 2, 90, {'s_id': 10}],
 6: [7, 7, 27, {'s_id': 2}],
 7: [12, 13, 27, {'s_id': 8}],
 8: [19, 17, 28, {'s_id': 14}],
 9: [19, 17, 28, {'s_id': 14}],
 10: [12, 13, 18, {'s_id': 8}],
 11: [19, 16, 31, {'s_id': 12}],
 12: [17, 17, 28, {'s_id': 14}]}

In [ ]:
2: [15, 16, 22, {'s_id': 12}],
10: [12, 13, 18, {'s_id': 8}],
6: [7, 7, 27, {'s_id': 2}],
7: [12, 13, 27, {'s_id': 8}],

In [186]:
#MC
# SSLP calcule bad in MC
d_normal_sspl_MC = []
len_path = {}
duibi = [result_MC_noraml[2],result_MC_noraml[10],result_MC_noraml[6],result_MC_noraml[7]]
t = [truth[12],truth[8],truth[2],truth[8]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_normal_sspl_MC.append(editdistance.eval(str(e_c), str(t[i_c]))-len(t[i_c]))
print(d_normal_sspl_MC)
print("d_normal_sspl_MC:",sum([27/15.5/40*10, 36/12.5/40*10, 28/7/40*10,34/12.5/40*10])/4)

[27, 36, 28, 34]
d_normal_sspl_MC: 0.7088709677419355


In [188]:
#SPM
# SSLP calcule normal in SPM
d_normal_sspl_SPM = []
len_path = {}
duibi = [result_SPM_noraml[2],result_SPM_noraml[10],result_SPM_noraml[6],result_SPM_noraml[7]]
t = [truth[12],truth[8],truth[2],truth[8]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_normal_sspl_SPM.append(editdistance.eval(str(e_c), str(t[i_c]))-len(t[i_c]))
print(d_normal_sspl_SPM)
print("d_normal_sspl_SPM:",sum([37/15.5/40*10, 43/12.5/40*10, 24/7/40*10,43/12.5/40*10])/4)

[37, 43, 24, 43]
d_normal_sspl_SPM: 0.793479262672811


### 这里是好学生的SSLP

In [196]:
#MC
# SSLP calcule bad in MC
distance = {}
len_path = {}
truth = l_good_novide
for i_c,e_c in enumerate(l_good_novide):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    for s_id,j in enumerate(truth): # truth grand
        d = editdistance.eval(str(e_c), str(j))
        if (d<=min_d) & (d!=0):
            min_d = d
            distance[i_c]=[(len(e_c)),len(j),int(d/(len(e_c)/10)), {"s_id":s_id}]
            len_path[i_c] = [e_c,j]
print("profile select for BAD")
distance

profile select for BAD


{0: [5, 5, 42, {'s_id': 11}],
 1: [32, 20, 45, {'s_id': 7}],
 2: [7, 5, 32, {'s_id': 11}],
 3: [94, 32, 56, {'s_id': 1}],
 4: [9, 5, 43, {'s_id': 0}],
 5: [17, 17, 40, {'s_id': 14}],
 6: [2, 2, 35, {'s_id': 10}],
 7: [20, 16, 24, {'s_id': 12}],
 8: [13, 16, 32, {'s_id': 12}],
 9: [13, 13, 40, {'s_id': 8}],
 10: [2, 2, 35, {'s_id': 6}],
 11: [5, 5, 42, {'s_id': 0}],
 12: [16, 17, 13, {'s_id': 14}],
 13: [11, 13, 40, {'s_id': 8}],
 14: [17, 16, 12, {'s_id': 12}]}

In [ ]:
这里是similar学生的匹对结果
12: [16, 17, 13, {'s_id': 14}]
14: [17, 16, 12, {'s_id': 12}]
7: [20, 16, 24, {'s_id': 12}]
11: [5, 5, 42, {'s_id': 0}],

In [208]:
#MC
# SSLP calcule bad in MC
d_good_sspl_MC = []
len_path = {}
duibi = [result_MC_good[12],result_MC_good[14],result_MC_good[7],result_MC_good[11]]
t = [truth[14],truth[12],truth[12],truth[0]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_good_sspl_MC.append(editdistance.eval(str(e_c), str(t[i_c]))-len(t[i_c]))
print(d_good_sspl_MC)
print("d_good_sspl_MC:",sum([34/16.5/40*10, 53/16.5/40*10, 61/18/40*10,20/6.5/40*10])/4)

[34, 53, 61, 20]
d_good_sspl_MC: 0.7336587024087025


In [201]:
#SPM
# SSLP calcule bad in SPM
d_good_sspl_SPM = []
len_path = {}
duibi = [result_SPM_good[12],result_SPM_good[14],result_SPM_good[7],result_SPM_good[2]]
t = [truth[14],truth[12],truth[12],truth[11]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_good_sspl_SPM.append(editdistance.eval(str(e_c), str(t[i_c]))-len(t[i_c]))
print(d_good_sspl_SPM)
print("d_good_sspl_SPM:",sum([49/16.5/40*10, 52/16.5/40*10, 29/6.5/40*10])/3)

[49, 52, 79, 29]
d_good_sspl_SPM: 0.8818958818958819


## DLP计算开始

In [221]:
#MC
# SSLP calcule bad in MC
distance = {}
len_path = {}
truth = l_normal_novide
for i_c,e_c in enumerate(l_bad_novide):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    for s_id,j in enumerate(truth): # truth grand
        d = editdistance.eval(str(e_c), str(j))
        if (d<=min_d) & (d!=0):
            min_d = d
            distance[i_c]=[(len(e_c)),len(j),int(d/(len(e_c)/10)), {"s_id":s_id}]
            len_path[i_c] = [e_c,j]
print("profile select for B,A")
distance

profile select for B,A


{0: [10, 7, 36, {'s_id': 6}],
 1: [39, 37, 40, {'s_id': 0}],
 2: [53, 37, 31, {'s_id': 0}],
 3: [6, 4, 41, {'s_id': 4}],
 4: [11, 12, 18, {'s_id': 10}],
 5: [1, 1, 50, {'s_id': 5}],
 6: [39, 37, 40, {'s_id': 0}],
 7: [11, 12, 18, {'s_id': 10}],
 8: [1, 4, 260, {'s_id': 4}],
 9: [10, 7, 36, {'s_id': 6}],
 10: [1, 4, 260, {'s_id': 4}],
 11: [1, 1, 50, {'s_id': 5}],
 12: [15, 15, 22, {'s_id': 2}],
 13: [25, 19, 34, {'s_id': 9}]}

In [219]:
#MC
# SSLP calcule bad in MC
distance = {}
len_path = {}
truth = l_good_novide
for i_c,e_c in enumerate(l_bad_novide):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    for s_id,j in enumerate(truth): # truth grand
        d = editdistance.eval(str(e_c), str(j))
        if (d<=min_d) & (d!=0):
            min_d = d
            distance[i_c]=[(len(e_c)),len(j),int(d/(len(e_c)/10)), {"s_id":s_id}]
            len_path[i_c] = [e_c,j]
print("profile select for B,G")
distance

profile select for B,G


{0: [10, 7, 34, {'s_id': 2}],
 1: [39, 32, 41, {'s_id': 1}],
 2: [53, 32, 41, {'s_id': 1}],
 3: [6, 5, 33, {'s_id': 11}],
 4: [11, 13, 29, {'s_id': 8}],
 5: [1, 2, 100, {'s_id': 6}],
 6: [39, 32, 41, {'s_id': 1}],
 7: [11, 13, 29, {'s_id': 8}],
 8: [1, 2, 90, {'s_id': 10}],
 9: [10, 7, 34, {'s_id': 2}],
 10: [1, 2, 90, {'s_id': 10}],
 11: [1, 2, 100, {'s_id': 6}],
 12: [15, 17, 19, {'s_id': 14}],
 13: [25, 20, 32, {'s_id': 7}]}

In [ ]:
G:12: [15, 17, 19, {'s_id': 14}],
    7: [11, 13, 29, {'s_id': 8}],
    4: [11, 13, 29, {'s_id': 8}],

In [235]:
#SPM
# SSLP calcule bad in SPM
d_bad_DLP_SPM = []
len_path = {}
duibi = [result_SPM_bad[12],result_SPM_bad[7],result_SPM_bad[4]]
t = [truth[14],truth[8],truth[8]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_bad_DLP_SPM.append(editdistance.eval(str(e_c), str(t[i_c]))-len(t[i_c]))
print(d_bad_DLP_SPM)
print("d_good_sspl_SPM:",sum([52/16.5/40*10, 56/12/40*10, 56/12.5/40*10])/3)

[52, 56, 56]
d_good_sspl_SPM: 1.024848484848485


In [237]:
#SPM
# SSLP calcule bad in SPM
d_bad_DLP_SPM = []
len_path = {}
duibi = [result_SPM_bad[12],result_SPM_bad[7],result_SPM_bad[4]]
t = l_normal_novide
t = [truth[2],truth[10],truth[10],truth[11]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_bad_DLP_SPM.append(editdistance.eval(str(e_c), str(t[i_c]))-len(t[i_c]))
print(d_bad_DLP_SPM)
print("d_good_sspl_SPM:",sum([49/16.5/40*10, 47/12/40*10, 47/12.5/40*10])/3)

[71, 82, 82]
d_good_sspl_SPM: 0.8871969696969697


In [224]:
1.024-0.887

0.137

In [243]:
#MC
# SSLP calcule bad in SPM
d_bad_DLP_MC = []
len_path = {}
t = l_normal_novide
duibi = [result_MC_bad[12],result_MC_bad[7],result_MC_bad[4]]
t = [truth[14],truth[8],truth[8]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_bad_DLP_MC.append(editdistance.eval(str(e_c), str(t[i_c]))-len(t[i_c]))
print(d_bad_DLP_MC)
print("d_normal_sspl_MC:",sum([63/16.5/40*10, 26/12/40*10, 23/12.5/40*10])/3)

[63, 26, 23]
d_normal_sspl_MC: 0.652070707070707


In [244]:
#MC
# SSLP calcule bad in SPM
d_bad_DLP_MC = []
len_path = {}
t = l_good_novide
duibi = [result_MC_bad[12],result_MC_bad[7],result_MC_bad[4]]
t = [truth[2],truth[10],truth[10]]
for i_c,e_c in enumerate(duibi):
    min_d = 10000
    #d2,p = fastdtw(e, e_c, dist=euclidean)
    d_bad_DLP_MC.append(editdistance.eval(str(e_c), str(t[i_c]))-len(t[i_c]))
print(d_bad_DLP_MC)
print("d_good_sspl_MC:",sum([58/16.5/40*10, 80/12/40*10, 76/12.5/40*10])/3)

[58, 80, 76]
d_good_sspl_MC: 1.3551515151515154


In [242]:
1.3-0.65

0.65

## 最后一种 PLP


*  ulmann2u  grebert5u 
*  poirot152u  marais8u  
*  marais8u   wolter6u 


### BAD vs GOOD+ average
* laghouat8u, schwab17u  
* marais8u braux4u;
* marais8u poirot152u;
---
### Average vs GOOD+average
* vignot1u  poirot152u
* lassalze3u courtin11u
* schwab17u  marchan72u  

In [250]:
for i,s in enumerate(s_good_novide):
    print(i,s,len(l_good_novide[i]))

0 wolter6u 5
1 bailly88u 32
2 marais8u 7
3 mogenot5u 94
4 baleste2u 9
5 braux4u 17
6 signer1u 2
7 poirot152u 20
8 grebert5u 13
9 ulmann2u 13
10 mekhilef3u 2
11 engler14u 5
12 weingart4u 16
13 dufour75u 11
14 oudart4u 17


In [339]:
print("GOOD students PLP start!!!")
#  ulmann2u  grebert5u 
#  poirot152u  marais8u  
#  marais8u   marais8u 
target = ["grebert5u","poirot152u","marais8u"]
t = [8,7,2]# length are 13, 20, 7
m = [8,2,1]
mentor = ["ulmann2u","marais8u","wolter6u "]
dis_MC = []
for i,s in enumerate(t):
    print(m[i],s)
    
    dis_MC.append(editdistance.eval(str(result_MC_good[s]), str(l_good_novide[m[i]])))
dis_SPM = []
for i,s in enumerate(t):
    dis_SPM.append(editdistance.eval(str(result_SPM_good[s]), str(l_good_novide[m[i]])))
print("MC for good in PLP",dis_MC)
print("SPM for good in PLP",dis_SPM)

GOOD students PLP start!!!
8 8
2 7
1 2
MC for good in PLP [43, 126, 196]
SPM for good in PLP [69, 115, 204]


In [340]:
print("MC result of PLP:",sum([43/13/7, 126/20/7])/2)
print("SPM result of PLP:",sum([69/13/7, 115/20/7])/2)

MC result of PLP: 0.6862637362637363
SPM result of PLP: 0.7898351648351648


#### Average
courtin11u  68.5186,58.3334  
lassalze3u  85.18520000000001,82.1429  
schwab17u  92.5926,78.5714  
marchan72u 68.5186,92.8571  

In [262]:
for i,s in enumerate(s_normal_novide+s_good_novide):
    print(i,s,len((l_normal_novide+l_good_novide)[i]))

0 vignot1u 37
1 schwab17u 9
2 courtin11u 15
3 pignolet3u 4
4 pignolet3u 4
5 jacobgui2u 1
6 marchan72u 7
7 pint3u 12
8 lassalze3u 19
9 lassalze3u 19
10 pirus7u 12
11 benkira12u 19
12 chelh1u 17
13 wolter6u 5
14 bailly88u 32
15 marais8u 7
16 mogenot5u 94
17 baleste2u 9
18 braux4u 17
19 signer1u 2
20 poirot152u 20
21 grebert5u 13
22 ulmann2u 13
23 mekhilef3u 2
24 engler14u 5
25 weingart4u 16
26 dufour75u 11
27 oudart4u 17


In [270]:
type(l_normal_novide+l_good_novide)

list

In [289]:
print("Average students PLP start!!!")
# * pignolet3u   engler14u 
# * lassalze3u courtin11u
# * schwab17u  marchan72u  
ll = l_normal_novide+l_good_novide
t = [4,8,1]# length are 13, 20, 7
m = [24,2,6]

dis_MC = []
for i,s in enumerate(t):
    print(editdistance.eval(str(result_MC_noraml[s]), str(ll[m[i]])))
    dis_MC.append(editdistance.eval(str(result_MC_noraml[s]), str(ll[m[i]])))
dis_SPM = []
for i,s in enumerate(t):
    dis_SPM.append(editdistance.eval(str(result_SPM_noraml[s]), str(ll[m[i]])))
print("MC for good in PLP",dis_MC)
print("SPM for good in PLP",dis_SPM)

GOOD students PLP start!!!
15
82
37
MC for good in PLP [15, 82, 37]
SPM for good in PLP [27, 88, 78]


In [324]:
print("MC result of PLP:",sum([15/4/7, 82/20/7])/2)
print("SPM result of PLP:",sum([22/4/7, 68/20/7])/2)

MC result of PLP: 0.5607142857142857
SPM result of PLP: 0.6357142857142857


In [316]:
for i,s in enumerate(s_bad_novide+s_good_novide+s_normal_novide):
    print(i,s,len((l_bad_novide+l_good_novide+l_normal_novide)[i])) 

0 mangeot38u 10
1 laghouat8u 39
2 schweize3u 53
3 maige1u 6
4 saran2u 11
5 lods3u 1
6 laghouat8u 39
7 saran2u 11
8 humber245u 1
9 mangeot38u 10
10 humber245u 1
11 lods3u 1
12 ladli2u 15
13 martin711u 25
14 wolter6u 5
15 bailly88u 32
16 marais8u 7
17 mogenot5u 94
18 baleste2u 9
19 braux4u 17
20 signer1u 2
21 poirot152u 20
22 grebert5u 13
23 ulmann2u 13
24 mekhilef3u 2
25 engler14u 5
26 weingart4u 16
27 dufour75u 11
28 oudart4u 17
29 vignot1u 37
30 schwab17u 9
31 courtin11u 15
32 pignolet3u 4
33 pignolet3u 4
34 jacobgui2u 1
35 marchan72u 7
36 pint3u 12
37 lassalze3u 19
38 lassalze3u 19
39 pirus7u 12
40 benkira12u 19
41 chelh1u 17


In [327]:
print("Bad students PLP start!!!")
# * mangeot38u  ,pirus7u   
# * humber245u pirus7u ;
# * ladli2u   marchan72u ;
ll = l_bad_novide+l_good_novide+l_normal_novide
t = [0,4,12]# length are 13, 20, 7
m = [39,39,31]

dis_MC = []
for i,s in enumerate(t):
    print(editdistance.eval(str(result_MC_bad[s]), str(ll[m[i]])))
    dis_MC.append(editdistance.eval(str(result_MC_bad[s]), str(ll[m[i]])))
dis_SPM = []
for i,s in enumerate(t):
    dis_SPM.append(editdistance.eval(str(result_SPM_bad[s]), str(ll[m[i]])))
print("MC for good in PLP",dis_MC)
print("SPM for good in PLP",dis_SPM)

Bad students PLP start!!!
39
31
64
MC for good in PLP [39, 31, 64]
SPM for good in PLP [46, 59, 64]


In [326]:
print("MC result of PLP bad:",sum([39/10/7, 33/10/7,64/15/7])/3)
print("SPM result of PLP bad:",sum([(36-10)/10/7, (61-10)/15/7],(70-10)/10/7)/3)

MC result of PLP bad: 0.546031746031746
SPM result of PLP bad: 0.5714285714285715


In [317]:
editdistance.eval(spm("mangeot38u"), str(ll[30]))

student is:  mangeot38u
unique pattern: ['321240,321240', '321240,327940', '321240,316750', '327940,321240', '327959,327960', '325068,327959', '316750,333191', '333191,327938', '327938,325068', '321240,321241', '321241,325071', '321240,333191', '327940,316750', '327938,327959', '316750,327938', '327940,330708', '325068,327960', '333191,325068', '325071,327940', '316751,321240', '321240,325071', '321240,327938', '316750,321240', '327940,333191', '325071,325072', '330708,333190', '316750,316751', '327938,327960', '325071,330708', '316750,325068', '325072,327940', '333191,327959', '321241,327940', '321241,325072', '327940,333190', '330708,330708', '327940,327941', '321240,325068', '327940,327938', '316751,321241', '327941,330708', '316750,327959', '316750,321241', '333191,327960', '325071,333190', '321240,330708', '321241,330708', '327940,327940', '316750,325071', '325072,330708', '321240,325072', '321240,327959', '327940,325068', '325072,327941', '325071,325071', '316751,325071', '316750

36

In [310]:
editdistance.eval(str(result_SPM_bad[6]), str(ll[30]))

217

In [307]:
ll[30]

[316750, 316750, 321240, 321240, 321240, 327940, 333191, 330708, 333190]

In [308]:
result_SPM_bad[1]

'321240,321241,325071,325072,327941,327941,333191,330708,333190,333191,327941,330708,333190,333191,327941,330708,333191,327941,333190,333191,327941,333190,333191,327941,327941,333191,330708,333191,327941,328647,327941,333191,327941,328647,327941,333191,327941,333191,327941'